# Room Type Recognition using YOLOv5

This notebook implements a room type recognition model using YOLOv5, a popular object detection model that we can adapt for image classification.

In [ ]:
# Install required packages
pip install nbformat install torch torchvision ultralytics matplotlib pandas seaborn scikit-learn

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

## Data Exploration

Let's check the distribution of our data across different room types.

In [ ]:
# Path to dataset
DATA_PATH = "../kaggle_room_street_data/house_data"

# Class names
class_names = os.listdir(DATA_PATH)
class_names = [c for c in class_names if not c.startswith(".")]
print(f"Classes: {class_names}")

# Count images per class
class_counts = {}
for class_name in class_names:
    class_dir = os.path.join(DATA_PATH, class_name)
    image_files = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    class_counts[class_name] = len(image_files)

# Print and visualize class distribution
print("
Class distribution:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count} images")

plt.figure(figsize=(10, 6))
plt.bar(class_counts.keys(), class_counts.values())
plt.xlabel("Room Type")
plt.ylabel("Number of Images")
plt.title("Images per Room Type")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Visualize Sample Images

In [ ]:
import matplotlib.image as mpimg

plt.figure(figsize=(15, 12))
for i, class_name in enumerate(class_names):
    class_dir = os.path.join(DATA_PATH, class_name)
    image_files = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    if image_files:
        sample_image = os.path.join(class_dir, random.choice(image_files))
        img = mpimg.imread(sample_image)
        plt.subplot(2, 3, i+1)
        plt.imshow(img)
        plt.title(class_name)
        plt.axis("off")

plt.tight_layout()
plt.show()

## Prepare Data for YOLOv5

YOLOv5 expects data in a specific format. Let's organize our data accordingly.

In [ ]:
# Create YOLOv5 dataset structure
YOLO_DATASET_PATH = "./room_yolo_dataset"
os.makedirs(YOLO_DATASET_PATH, exist_ok=True)

# Create train, val, test directories
for split in ["train", "val", "test"]:
    for subdir in ["images", "labels"]:
        os.makedirs(os.path.join(YOLO_DATASET_PATH, split, subdir), exist_ok=True)

# Create file lists for each class
all_images = []
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}

for class_name in class_names:
    class_dir = os.path.join(DATA_PATH, class_name)
    image_files = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    
    for img_file in image_files:
        all_images.append({
            "class": class_name,
            "class_idx": class_indices[class_name],
            "path": os.path.join(class_dir, img_file),
            "filename": img_file
        })

# Split data into train (70%), validation (15%), and test (15%) sets
train_imgs, temp_imgs = train_test_split(all_images, test_size=0.3, random_state=42, stratify=[img["class"] for img in all_images])
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42, stratify=[img["class"] for img in temp_imgs])

print(f"Train images: {len(train_imgs)}")
print(f"Validation images: {len(val_imgs)}")
print(f"Test images: {len(test_imgs)}")

In [ ]:
# Function to prepare YOLO format data
def prepare_yolo_data(image_list, split):
    for img_data in image_list:
        # Copy image to YOLO dataset
        src_path = img_data["path"]
        dst_path = os.path.join(YOLO_DATASET_PATH, split, "images", img_data["filename"])
        shutil.copy(src_path, dst_path)
        
        # Create corresponding label file (since we are doing classification, we will use a dummy bbox)
        label_file = os.path.join(YOLO_DATASET_PATH, split, "labels", os.path.splitext(img_data["filename"])[0] + ".txt")
        
        # Format: class_idx center_x center_y width height
        # For classification, we will use a full-image bbox (centered at 0.5, 0.5 with width/height of 1.0)
        with open(label_file, "w") as f:
            f.write(f"{img_data["class_idx"]} 0.5 0.5 1.0 1.0
")

# Prepare data for each split
prepare_yolo_data(train_imgs, "train")
prepare_yolo_data(val_imgs, "val")
prepare_yolo_data(test_imgs, "test")

In [ ]:
# Create data.yaml file for YOLOv5
yaml_content = f"""train: {os.path.join(YOLO_DATASET_PATH, "train", "images")}
val: {os.path.join(YOLO_DATASET_PATH, "val", "images")}
test: {os.path.join(YOLO_DATASET_PATH, "test", "images")}

nc: {len(class_names)}
names: {class_names}
"""

with open(os.path.join(YOLO_DATASET_PATH, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("YOLO dataset preparation complete.")